# Comparing networks

In [ ]:
import Pkg; Pkg.activate(".")
using EcologicalNetworks
using EcologicalNetworksPlots
using Plots

For this example, we will work on the data from the host-parasite study in Eurasia -- there are 51 unique networks, and we want all of them converted to binary networks as we do not account for interaction strength.

In [ ]:
HP = convert.(BipartiteNetwork, web_of_life.("A_HP_0".*lpad.(1:51, 2, "0")))

For a first illustration, we will get a pair of networks

In [ ]:
N1 = HP[3]
N2 = HP[14]

We can *merge* these networks, to create what we call a *metaweb* -- this is the aggregation of all species and all interactions:

In [ ]:
metaweb = union(N1, N2)

We will start by plotting these networks, to see what they look like -- we will plot the metaweb, then overlay network 1, then network 2:

In [ ]:
I = initial(RandomInitialLayout, metaweb)
for step in 1:4000
  position!(ForceDirectedLayout(), I, metaweb)
end

In [ ]:
plot(I, metaweb, lc=:lightgrey)
scatter!(I, metaweb, mc=:lightgrey, msw=0)

In [ ]:
plot!(I, N1, lc=:blue)
scatter!(I, N1, mc=:blue, msw=0)

In [ ]:
plot!(I, N2, lc=:red)
scatter!(I, N2, mc=:red, msw=0)

We can now plot the elements that are shared by both networks:

In [ ]:
plot(I, metaweb, lc=:lightgrey)
scatter!(I, metaweb, mc=:lightgrey, msw=0)
plot!(I, intersect(N1, N2), lc=:purple, lw=3)
scatter!(I, intersect(N1, N2), mc=:purple, msc=:purple, msw=3)

## Measuring dissimilarity

We can now measure the number of elements shared by the two networks, by either counting species, interactions, or the interactions between shared species:

In [ ]:
βwn(N1, N2)

In [ ]:
βos(N1, N2)

In [ ]:
βs(N1, N2)

With these information, we can measure the *pairwise dissimilarity* between these networks.

In [ ]:
KGL02(βos(N1, N2))

Now let's do something more general, and compare all pairs of networks. Because we are using a symmetrical measure, we only need to do $\frac{1}{2}n(n-1)$ comparisons:

In [ ]:
n = length(HP)
os = zeros(Float64, convert(Int64, n*(n-1)/2))
bs = zeros(Float64, convert(Int64, n*(n-1)/2))
wn = zeros(Float64, convert(Int64, n*(n-1)/2))

f = KGL02

cursor = 1
for i in 1:(n-1), j in (i+1):n
    os[cursor] = f(βos(HP[i], HP[j]))
    bs[cursor] = f(βs(HP[i], HP[j]))
    wn[cursor] = f(βwn(HP[i], HP[j]))
    global cursor += 1
end

In [ ]:
scatter(bs, wn, leg=false, xlab="Species dissimilarity", ylab="Network dissimilarity", aspectratio=1)
yaxis!((0,1))
xaxis!((0,1))

In [ ]:
scatter(bs, os, leg=false, xlab="Species dissimilarity", ylab="Interactions dissimilarity", aspectratio=1)
yaxis!((0,1))
xaxis!((0,1))

## A measure of network originality

We can also measure how much interactions differ between every network and the regional *metaweb*, using $\beta_{os}'$:

In [ ]:
M = reduce(union, HP)
partitions = [βos(M, hp) for hp in HP]
osprime = f.(partitions)
histogram(osprime, leg=false)
xaxis!("Dissimilarity to metaweb", (0,1))

This shows that while networks are on average *not too dissimilar* (about 0.3), some of them can have a lot fewer interactions than we expect. We may want to see whether this is linked to their connectance:

In [ ]:
scatter(connectance.(HP), osprime, leg=false)
xaxis!((0,1), "Connectance")
yaxis!((0,1), "Dissimilarity to metaweb")